In [1]:
import os
from tqdm.notebook import tqdm
from rdflib import Graph, Namespace, RDF

os.chdir("..")

from scripts.annotations import AnnotationQuestion, AnnotationOfInstance, AnnotationOfRelation, AnnotationOfAnswerSPARQL
from scripts.functions import parse_component, reachable, find_output_annotation_in_combination

data_dir = "./data/"
descriptions_dir = os.path.join(data_dir, "component-descriptions")

# Define the namespaces used in the RDF/Turtle file
QA = Namespace('https://w3id.org/wdaqua/qanary#')
RDFS = Namespace('http://www.w3.org/2000/01/rdf-schema#')

## Parse RDF-descriptions into Python objects

In [2]:
description_files = os.listdir(descriptions_dir) # load all rdf description file names in the directory

In [3]:
components = []

for file in description_files: # iterate over available component descriptions

    # Load the RDF/Turtle file into an rdflib graph
    try:
        g = Graph()
        with open(os.path.join(descriptions_dir, file), 'r') as f:
            g.parse(f, format='turtle')

        # Find the component type
        component_type = [t for t in g.triples((QA[file.replace('.ttl', '')], RDF.type, None))][0][2].toPython()
        # Find the component in the graph
        component_uri = QA[file.replace('.ttl', '')]
        component = parse_component(g, component_uri, component_type)
        components.append(component)
    except Exception as e:
        print(file)
        print(e)

In [4]:
print("The following components are there:", components)

The following components are there: [PythonMBart, OpentapiocaNED, TextRazorNER, TagMeNED, CLSNLIOD, AmbiverseNED, MeaningCloudNER, StanfordNER, LDShuyo, ComicCharacterNameSimpleNER, MeaningCloudNED, PythonLibreTranslate, DBpediaSpotlightNED, PythonNLLB, BabelfyNED, EntityClassifier2NER, Sina, PythonHelsinkiNLP, WatsonNED, AGDISTISNED, ReMatch, QAnswer, DandelionNER, AylienNER, DBpediaSpotlightNER, DandelionNED, OntotextNER, BabelfyNER, AylienNED, FoxNER, AmbiverseNER, TagMeNER, OntotextNED]


## Create all the possible combinations of the components

### 1. Permutation of types

How many ways can you place $n$ different objects in $k$ different places ($k$ $\in$ $1,2,\ldots,n$)?

\begin{equation}
P^k_n = \frac{n!}{(n-k)!}
\end{equation}

Where $n$ is a number of types and $k$ is a number of abstract components in a sequence.

In [5]:
from itertools import permutations

In [6]:
type_component_dict = {}

# construct a dictionary mapping component types to actual components that have that type
for component in components:
    type_component_dict[type(component)] = type_component_dict.get(type(component), []) + [component]

In [7]:
len_permutations_dict = {}

# construct a dictionary mapping the length of a permutation to all possible permutations of component types
for k in range(1, len(type_component_dict.keys()) + 1):
    len_permutations_dict[k] = [p for p in permutations(type_component_dict.keys(), k)]

### 2. Permutation of components within a permutation of types

Let $k$ actions be performed sequentially. If the first action can be performed in $n_1$ ways, the second action in $n_2$ ways, the third action in $n_3$ ways, and so on to the $k$-th. Then the total number of ways in which the $k$ actions can be performed is:

\begin{equation}
N = n_1*n_2*...*n_k
\end{equation}

Where $n_i$ is a number of actual components of the $i$-th type (e.g., NER, REL, QB).

In [8]:
def combine_lists(lists):
    """
    Combines the provided list of lists into a list of tuples, where each tuple contains a component combination.
    N = n_1*n_2*...*n_k, where n_i is the length of the i-th list in lists and k is the length of the lists.

    Args:
        lists (_type_): a list of lists of actual components

    Returns:
        list: a list of tuples of actual components (combinations)
    """
    if not lists:
        return []
    elif len(lists) == 1:
        return [(x,) for x in lists[0]]
    else:
        result = []
        for item in lists[0]:
            for subitem in combine_lists(lists[1:]):
                result.append((item,) + subitem)
        return result

In [9]:
combinations = []
for length, abstract_perm in len_permutations_dict.items():
    for perm in abstract_perm:
        component_type_lists = [type_component_dict[component_type] for component_type in perm]
        combinations += combine_lists(component_type_lists)

In [10]:
len(combinations)

9956021

## Mapping the combinations to Petri Net semantics with SNAKE

In [11]:
from snakes.nets import *

BATCH_SIZE = 3 * 10 ** 4

In [12]:
def create_nets_batch(combinations: list):
    nets = []

    for combination in combinations: # a single combination refers to a single petri net
        n = PetriNet(' --> '.join(str(c) for c in combination))
        
        n.add_place(Place('questionPlace', [1, 'http://id.loc.gov/vocabulary/iso639-1/en'])) # assume english question on start

        for component in combination:
            place_name = str(component) + 'Place'
            transition_name = str(component) + 'Transition'
            n.add_place(Place(place_name, []))
            n.add_transition(Transition(name=transition_name, guard=Expression(' and '.join([a.get_guard_expression() for a in component.input_annotations]))))
            
        prev_variables = []
        for i in range(len(combination)):
            input_variables = list(set([v for ia in combination[i].input_annotations for v in ia.get_input_variables()]))
            input_variables = [v for v in prev_variables if v not in input_variables and type(v) != Value] + input_variables # add those prev_variables that are not in input_variables
            output_values = list(set([v for oa in combination[i].output_annotations for v in oa.get_output_values()]))
            
            n.add_input(n.place()[i].name, n.transition()[i].name, MultiArc(input_variables))
            n.add_output(n.place()[i + 1].name, n.transition()[i].name, MultiArc(input_variables + output_values))
            
            output_variables = list(set([v for oa in combination[i].output_annotations for v in oa.get_output_variables()]))
            prev_variables = input_variables + output_variables

        nets.append(n)
    
    return nets

## Produce the reachable combinations

In [13]:
for i in tqdm(range(0, len(combinations), BATCH_SIZE)): # we use a batch size to avoid memory issues
    if i + BATCH_SIZE > len(combinations):
        nets = create_nets_batch(combinations[i:])
    else:
        nets = create_nets_batch(combinations[i:i + BATCH_SIZE])
    
    for i in tqdm(range(len(nets))):
        annotation = AnnotationOfAnswerSPARQL # search for this as a produced token in the net
        if find_output_annotation_in_combination(annotation, combinations[i]): # theoretical possibility that the annotation is in the combination
            if reachable(nets[i], annotation.token_value):
                print(nets[i], "Success!")
    
    del nets

  0%|          | 0/332 [00:00<?, ?it/s]

  0%|          | 0/30000 [00:00<?, ?it/s]

QAnswer Success!
PythonMBart --> QAnswer Success!
PythonNLLB --> QAnswer Success!
PythonHelsinkiNLP --> QAnswer Success!
OpentapiocaNED --> QAnswer Success!
TagMeNED --> QAnswer Success!
AmbiverseNED --> QAnswer Success!
MeaningCloudNED --> QAnswer Success!
DBpediaSpotlightNED --> QAnswer Success!
BabelfyNED --> QAnswer Success!
WatsonNED --> QAnswer Success!
DandelionNED --> QAnswer Success!
AylienNED --> QAnswer Success!
OntotextNED --> QAnswer Success!
TextRazorNER --> QAnswer Success!
MeaningCloudNER --> QAnswer Success!
StanfordNER --> QAnswer Success!
ComicCharacterNameSimpleNER --> QAnswer Success!
EntityClassifier2NER --> QAnswer Success!
DandelionNER --> QAnswer Success!
AylienNER --> QAnswer Success!
DBpediaSpotlightNER --> QAnswer Success!
OntotextNER --> QAnswer Success!
BabelfyNER --> QAnswer Success!
FoxNER --> QAnswer Success!
AmbiverseNER --> QAnswer Success!
TagMeNER --> QAnswer Success!
CLSNLIOD --> QAnswer Success!
LDShuyo --> QAnswer Success!
QAnswer --> PythonMBart

### Tests / Debugging

In [ ]:
test_combinations = []

for i in range(len(combinations)):
    comb_str = ' --> '.join(str(c) for c in combinations[i])
    components = ["DBpediaSpotlight", "ReMatch", "Sina"]
    
    if all(c in comb_str for c in components):
        test_combinations.append(combinations[i])
        print(comb_str)

In [ ]:
test_combinations[3]

In [ ]:
combination = test_combinations[3]

n = PetriNet(' --> '.join(str(c) for c in combination))
# define places
n.add_place(Place('questionPlace', [1, 'https://w3id.org/wdaqua/qanary/English']))

for component in combination:
    place_name = str(component) + 'Place'
    transition_name = str(component) + 'Transition'
    n.add_place(Place(place_name, []))
    n.add_transition(Transition(name=transition_name, guard=Expression(' and '.join([a.get_guard_expression() for a in component.input_annotations]))))
    
prev_variables = []
for i in range(len(combination)):
    input_variables = list(set([v for ia in combination[i].input_annotations for v in ia.get_input_variables()]))
    input_variables = [v for v in prev_variables if v not in input_variables and type(v) != Value] + input_variables # add those prev_variables that are not in input_variables
    output_values = list(set([v for oa in combination[i].output_annotations for v in oa.get_output_values()]))
    
    n.add_input(n.place()[i].name, n.transition()[i].name, MultiArc(input_variables))
    n.add_output(n.place()[i + 1].name, n.transition()[i].name, MultiArc(input_variables + output_values))
    
    output_variables = list(set([v for oa in combination[i].output_annotations for v in oa.get_output_variables()]))
    prev_variables = input_variables + output_variables

In [ ]:
marking_value = AnnotationOfAnswerSPARQL.token_value

n = test_nets[3]

for transition in n.transition():
    if len(transition.modes()) == 0: # deadlock
        break
    
    transition.fire(transition.modes()[0])

    for t in n.get_marking().keys():
        if marking_value in n.get_marking()[t].items():
            break